# Étude d'un circuit RC avec Arduino

Ce notebook permet de suivre l'évolution de la tension aux bornes d'un condensateur après une modification de la tension délivrée par une voie numérique. Il utilise directement le code `acquisition_continue.ino` téléversé sur la carte Arduino.

In [ ]:
# Import des bibliothèques nécessaires
%run /home/laurent/fichiers_python/Import_bibliotheques.ipynb
%run /home/laurent/fichiers_python/Arduino.ipynb

## Conditions expérimentales :

In [ ]:
charge = 1    # mettre 1 pour la charge du condensateur, et 0 pour sa décharge
duree = 0.2    # Entrer la durée d'acquisition désirée en s
Analog_pin = 0    # numéro de la voie de mesure analogique (déjà configurée dans le code Arduino)
Digital_pin = 2    # numéro de la voie de commande digitale (doit être entre 2 et 5 pour notre code Arduino)

## Connexion à Arduino et acquisition des données

In [ ]:
# Initialisation des listes pour stocker les données
uc = []
t = []

# Détection et connexion à l'Arduino
try:
    connexion_arduino()
    print("Connexion établie avec Arduino")
except Exception as e:
    print(f"Erreur de connexion: {e}")
    raise

In [ ]:
# Fonction pour basculer une broche digitale
def set_digital_pin(pin, state):
    command = f"D{pin}:{state}\n"
    ser.write(command.encode())
    response = ser.readline().decode().strip()
    print(f"Réponse Arduino: {response}")

In [ ]:
# Préparation de l'acquisition
print(f"Condition initiale: {'charge' if charge == 1 else 'décharge'}")

# Mise en place de la condition initiale (inverse de ce qu'on va faire ensuite)
set_digital_pin(Digital_pin, 1 - charge)
sleep(0.5)  # Attente pour stabilisation

# Vidage du buffer série
ser.reset_input_buffer()

# Démarrage de l'acquisition
print(f"Début de l'acquisition pour {duree} secondes...")
debut = time()

# Basculement de la broche digitale pour démarrer la charge/décharge
set_digital_pin(Digital_pin, charge)

# Acquisition des données pendant la durée spécifiée
while time() - debut < duree:
    try:
        ligne = ser.readline().decode('utf-8').strip()
        if ligne and ',' in ligne:
            valeurs = ligne.split(',')
            if len(valeurs) == 2:
                temps_ms = int(valeurs[0])
                tension = float(valeurs[1])
                if len(t) == 0:
                    temps_initial = temps_ms
                temps_relatif = (temps_ms - temps_initial) / 1000.0  # Conversion en secondes
                t.append(temps_relatif)
                uc.append(tension)
    except Exception as e:
        print(f"Erreur lors de la lecture: {e}")
        continue

print(f"Acquisition terminée. {len(t)} points collectés.")

# Conversion des listes en tableaux numpy
t = array(t)
uc = array(uc)

## Calcul des caractéristiques de la courbe de régression

In [ ]:
# Définition des contraintes pour le modèle
if charge == 0:
    equation = "E * exp(-1 * t / tau)"
else:
    equation = "E * (1 - exp(-1 * t / tau))"

# Contraintes sous forme de liste de tuples (nom_paramètre, valeur_initiale, vary, min, max)
contraintes = [
    ('tau', 1, True, 0, inf),
    ('E', 5, True, 0, 5.5)
]

# Calcul du modèle
xMod, yMod, expression, valeurs, abscisse, ordonnee, modele, parametres = Calcul_modele(
    't',           # abscisse
    'uc',          # ordonnée
    equation,      # équation
    0,             # début de la plage de données à utiliser
    len(t),        # fin de la plage de données à utiliser
    None,          # début de la courbe du modèle (None = min des données)
    None,          # fin de la courbe du modèle (None = max des données)
    contraintes    # contraintes sur les paramètres
)

## Tracé du graphique

In [ ]:
figure(figsize=(10, 6))

# Tracé des points expérimentaux :
plot(abscisse, ordonnee, '+', label="Points expérimentaux")

# Tracé du modèle :
plot(xMod, yMod, '-', label=f"Modèle :    {expression} \n {valeurs}")

grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Variation de la tension aux bornes du condensateur")
xlabel("t (s)")
ylabel("uc (V)")
show()

## Calcul de la valeur de la capacité du condensateur

In [ ]:
R = 220    # Entrer la valeur de la résistance en Ohm
c = tau / R
print(f"La valeur de la capacité du condensateur est {c:.3g} F")

In [ ]:
# Fermeture de la connexion série
ser.close()
print("Connexion avec Arduino fermée.")